In [1]:
import pandas as pd

from sklearn.model_selection import train_test_split

#from gensim.models import word2vec

from nltk.tokenize import RegexpTokenizer
from nltk import word_tokenize

#tokenize df.sentence
def tokenize(strArr):
    tokenizedArr=[]
    for curStr in strArr:
        tokenizedArr.append(word_tokenize(curStr))
    return tokenizedArr

#input dataset should be an array of lists of strings
#max lenth = 55
#take each word in the thyroid sentence and return an integer based on the word2vec model
def convertToEmbeddingInts(model, inputDs, maxLen):
    mykeys=model.wv.vocab.keys()
    indBig=0
    for curstr in inputDs:
        embedding=np.zeros(len(curstr))
        ind=0
        for tok in curstr:
            if tok in mykeys:
                embedding[ind]=list(mykeys).index(tok)+1
            else:
                embedding[ind]=0
            ind=ind+1
        if len(embedding)>maxLen:
            embedding=embedding[:maxLen]
        inputDs[indBig]=embedding
        indBig=indBig+1
    inputDs=keras.preprocessing.sequence.pad_sequences(inputDs, maxlen=maxLen)
    return inputDs

#output array of integers of uniform length



In [2]:
#keras fast text method
df = pd.read_csv('10k_bi.csv')
df = df.iloc[:,0:2]
df.dropna()
text2 = df.sentence.str.replace("\[", "")

text3 = text2.str.replace("\]", "")
text4 = text3.str.replace("'", "")
text4.head()

0                   the thyroid gland is unremarkable.
1    the thyroid gland and esophagus are unremarkable.
2    the thyroid gland and esophagus are unremarkable.
3                   the thyroid gland is unremarkable.
4                   findings: thyroid gland is normal.
Name: sentence, dtype: object

In [3]:
import nltk
#nltk.download('punkt')

In [4]:
df.sentence = text4
tokenizedarray = tokenize(df.sentence)
print(type(tokenizedarray))


<class 'list'>


In [5]:
import numpy as np
df2 = pd.DataFrame(np.array(tokenizedarray))
df2.head()


,0
0,"[the, thyroid, gland, is, unremarkable, .]"
1,"[the, thyroid, gland, and, esophagus, are, unr..."
2,"[the, thyroid, gland, and, esophagus, are, unr..."
3,"[the, thyroid, gland, is, unremarkable, .]"
4,"[findings, :, thyroid, gland, is, normal, .]"


In [6]:
df2.columns = ['toks']
df2.head()

,toks
0,"[the, thyroid, gland, is, unremarkable, .]"
1,"[the, thyroid, gland, and, esophagus, are, unr..."
2,"[the, thyroid, gland, and, esophagus, are, unr..."
3,"[the, thyroid, gland, is, unremarkable, .]"
4,"[findings, :, thyroid, gland, is, normal, .]"


In [7]:
df3 = pd.concat([df, df2], axis=1)
df3.head()

,sentence,class,toks
0,the thyroid gland is unremarkable.,0,"[the, thyroid, gland, is, unremarkable, .]"
1,the thyroid gland and esophagus are unremarkable.,0,"[the, thyroid, gland, and, esophagus, are, unr..."
2,the thyroid gland and esophagus are unremarkable.,0,"[the, thyroid, gland, and, esophagus, are, unr..."
3,the thyroid gland is unremarkable.,0,"[the, thyroid, gland, is, unremarkable, .]"
4,findings: thyroid gland is normal.,0,"[findings, :, thyroid, gland, is, normal, .]"


In [8]:
df3.tail()

,sentence,class,toks
6573,findings: the partially visualized thyroid is ...,0,"[findings, :, the, partially, visualized, thyr..."
6574,findings: the partially visualized thyroid is ...,0,"[findings, :, the, partially, visualized, thyr..."
6575,findings: mild asymmetric atrophy of the left ...,0,"[findings, :, mild, asymmetric, atrophy, of, t..."
6576,the thyroid is partially visualized and normal...,0,"[the, thyroid, is, partially, visualized, and,..."
6577,thyroid is normal.,0,"[thyroid, is, normal, .]"


In [9]:
from gensim.models import Word2Vec
model1 = Word2Vec.load('300dallchestnoncon')

In [10]:
long = []
for row in df3.toks:
    long.append(len(row))
np.max(long)

163

In [11]:
%%time
import keras.preprocessing.sequence
toks = df3.toks
ints = convertToEmbeddingInts(model1, toks, 163)

Using TensorFlow backend.
C:\wustl\envs\tf-gpu\lib\site-packages\ipykernel_launcher.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Wall time: 12 s


In [12]:
def makeEmbeddingMatrix(model):
    mykeys=model.wv.vocab.keys()
    embeddingMatrix=np.zeros((len(mykeys)+1,300))
    ind=1 #First entry is all zeros
    for key in mykeys:
        embeddingMatrix[ind]=model[key]
        ind=ind+1
    return embeddingMatrix

embed = makeEmbeddingMatrix(model1)

C:\wustl\envs\tf-gpu\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


In [13]:
X_train, X_test, y_train, y_test = train_test_split(ints,
                                                    pd.get_dummies(df3['class'], drop_first=False),
                                                    test_size=0.2,
                                                    stratify= df3['class'],
                                                    random_state=42)

In [14]:
X_train, X_val, y_train, y_val = train_test_split(X_train,
                                                    y_train,
                                                    test_size=0.25,
                                                    stratify= y_train,
                                                    random_state=42)

In [15]:
y_train = y_train.values
y_test = y_test.values
y_val = y_val.values

In [16]:
#KERAS MODEL DEFINITION
from keras.layers import Dense, Dropout, Embedding
from keras.layers import Flatten, Input, SpatialDropout1D, GlobalAveragePooling1D, Conv1D, LSTM
from keras.models import Model
from keras.optimizers import Adam 
from keras.regularizers import l1_l2
maxlen = 163
def get_model():
    embed_dim = 300
    dropout_rate = 0.5
    emb_dropout_rate = 0.5
   
    input_text = Input(shape=[maxlen], name="stem_input")
    
    emb = (Embedding(np.shape(embed)[0], np.shape(embed)[1], weights=[embed], trainable=False))(input_text)
                                                                
    lstm = LSTM(64, activation = 'relu', dropout = 0.2, recurrent_dropout = 0.2, kernel_regularizer = l1_l2(0.0001),
                implementation = 1, unroll = False)(emb)
    
    #pool = GlobalAveragePooling1D()(emb)
    
    #dense = Dropout(dropout_rate)(Dense(256, activation = 'relu')(pool))
    
    output = Dense(2, activation="softmax")(lstm)

    model = Model([input_text], output)

    optimizer = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
    model.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=["accuracy"])
    return model

model = get_model()
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
stem_input (InputLayer)      (None, 163)               0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 163, 300)          5157300   
_________________________________________________________________
lstm_1 (LSTM)                (None, 64)                93440     
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 130       
Total params: 5,250,870
Trainable params: 93,570
Non-trainable params: 5,157,300
_________________________________________________________________


In [17]:
%%time
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.models import load_model

modelFilePath = 'C:/Users/Ryan Short/Desktop/lstm10kv3.hdf5'
checkpoint = ModelCheckpoint(modelFilePath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')


epochs = 300
hist = model.fit(X_train, y_train,
                 batch_size=64,
                 validation_data=(X_val, y_val),
                 epochs=epochs,
                 callbacks=[EarlyStopping(patience=10, monitor='val_loss'), checkpoint])

Train on 3946 samples, validate on 1316 samples
Epoch 1/300
3946/3946 [==============================] - 18s 5ms/step - loss: 2.6419 - accuracy: 0.9673 - val_loss: 1.8896 - val_accuracy: 0.9772

Epoch 00001: val_loss improved from inf to 1.88965, saving model to C:/Users/Ryan Short/Desktop/lstm10kv3.hdf5
Epoch 2/300
3946/3946 [==============================] - 19s 5ms/step - loss: 1.4028 - accuracy: 0.9780 - val_loss: 1.0252 - val_accuracy: 0.9795

Epoch 00002: val_loss improved from 1.88965 to 1.02520, saving model to C:/Users/Ryan Short/Desktop/lstm10kv3.hdf5
Epoch 3/300
3946/3946 [==============================] - 20s 5ms/step - loss: 0.8107 - accuracy: 0.9828 - val_loss: 0.6604 - val_accuracy: 0.9840

Epoch 00003: val_loss improved from 1.02520 to 0.66044, saving model to C:/Users/Ryan Short/Desktop/lstm10kv3.hdf5
Epoch 4/300
3946/3946 [==============================] - 20s 5ms/step - loss: 0.7085 - accuracy: 0.9734 - val_loss: 1.0176 - val_accuracy: 0.9597

Epoch 00004: val_loss d

In [18]:
from keras.models import load_model
model = load_model(modelFilePath)
test_pred = model.predict(X_test, batch_size=64)

In [19]:
test_pred2 = test_pred.argmax(1)
y_test2 = y_test.argmax(1)

In [20]:
from sklearn import metrics
tn, fp, fn, tp = metrics.confusion_matrix(y_test2, test_pred2).ravel()
print('True Negative: ', tn)
print('False Positive: ', fp)
print('False Negative: ', fn)
print('True Positive: ', tp)

print('Accuracy: ', (tp+tn)/(tp+fp+fn+tn))
print('Precision/PPV: ', tp/(tp+fp))
print('Sensitivity/Recall: ', tp/(tp+fn))

True Negative:  1272
False Positive:  6
False Negative:  19
True Positive:  19
Accuracy:  0.9810030395136778
Precision/PPV:  0.76
Sensitivity/Recall:  0.5


In [21]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test2, test_pred2, average= 'macro')

0.8800140021415039